In [1]:
# set up logging
import logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -    %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [10]:
import math
from torch.utils.data import Dataset


class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print(f"data has {data_size:d} characters, {vocab_size:d} unique.")

        self.stoi = { ch: i for i, ch in enumerate(chars) }
        self.itos = { i: ch for i, ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx+self.block_size+1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [11]:
block_size = 128  # spacial extent of the model for its context

In [20]:
# text = open('/home/grads/xiaohan/scratch/minGPT/data/The Old Man and the Sea.txt', 'r').read()
text = open('/home/grads/xiaohan/scratch/minGPT/data/135.txt', 'r').read()
train_dataset = CharDataset(text, block_size = 128) # one line of poem is roughly 50 characters

data has 3250633 characters, 119 unique.


In [21]:
from mingpt.model import GPT, GPTConfig


mconf = GPTConfig(
    train_dataset.vocab_size,
    train_dataset.block_size,
    n_layer=4,
    n_head=8,
    n_embd=512,
)
model = GPT(mconf)

01/09/2023 18:58:28 - INFO - mingpt.model -    number of parameters: 12797952


In [22]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(
    max_epochs=2,
    batch_size=512,
    learning_rate=6e-4,
    lr_decay=True,
    warmup_tokens=512*20,
    final_tokens=2*len(train_dataset)*block_size,
    num_workers=4,
)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

  0%|                                                              | 0/6349 [00:00<?, ?it/s]/home/grads/xiaohan/scratch/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
epoch 1 iter 6348: train loss 0.70582, lr 3.000058e-04: 100%|█| 6349/6349 [10:14<00:00, 10.3
epoch 2 iter 6348: train loss 0.55222, lr 6.000000e-05: 100%|████████████| 6349/6349 [10:12<00:00, 10.36it/s]


In [27]:
# alright, let's sample some character-level Shaespeare
from mingpt.utils import sample

context = "Jean Valjean made his appearance"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

Jean Valjean made his appearance in advance of him, which was all
sorts of secrets for soldiers, and of which he would have been greatly
preserved the appearance of superstition, as in contemplation before he
had controlled himself since he had counted on a bickcaxe a spectre
of grape-shot had scaled his face; but he had no longer any need of saying
either “father” to him. A flash of joy illuminated his whole person
who was charged to release him. All these men that they had set out to
follow, nevertheless, only he had meant to take her revenge and had
called her thoughts.—“This are still bleeding.” She shouted to him:—

“Courfeyrac! Courfeyrac! Hohée! He was out! The old man who delivers himself
bonnet.”


The sewer in a severe space, seemed to be appointed the Bishop as
real as he was forced to sell, stepped out, but shutting at his eyes on
the stone, and then he perceived a light.

While the Bishop was behind the trees with the phantoms, as though
enthusiastic over this laughter of 